<h2> Get sequence <h2>

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import SpaDC 

def chr_split(x):
    chr_special = {'GL456233.1':'chrX_GL456233_random', 'GL456212.1':'chr1_GL456212_random', 'JH584304.1':'chrUn_JH584304', 'GL456216.1':'chr4_GL456216_random', 'JH584292.1':'chr4_JH584292_random', 'JH584295.1':'chr4_JH584295_random'}
    chr, start, end = x.split('-')
    if chr in chr_special:
        chr = chr_special[chr]
    return chr, start, end

misar = sc.read_h5ad('/home/nas2/biod/machuanlong/data/MISAR_seq/MISAR_seq_mouse_E15_brain_ATAC_data.h5ad')
ATAC = sc.read_h5ad('/home/nas2/biod/machuanlong/data/P22/ATAC/adata_peaks_normalized.h5ad')

<h3> MISAR <h3>

In [3]:
index = misar.var_names
index = pd.DataFrame(chr_split(x) for x in index)

seq, _ = SpaDC.make_bed_seqs_from_df(index, '/home/nas2/biod/machuanlong/data/Genome/mm10.fa', 1344)

file = open('/home/nas2/biod/machuanlong/data/MISAR_seq/seqs.txt','a')
file.write('seq\n')
for i in range(len(seq)):
    s = seq[i] + '\n'
    file.write(s)
file.close()
print("保存成功") 

保存成功


<h3> P22 <h3>

In [4]:
index = ATAC.var_names
index = pd.DataFrame(chr_split(x) for x in index)

seq, _ = SpaDC.make_bed_seqs_from_df(index, '/home/nas2/biod/machuanlong/data/Genome/mm10.fa', 1344)

file = open('/home/nas2/biod/machuanlong/data/P22/ATAC/seqs.txt','a')
file.write('seq\n')
for i in range(len(seq)):
    s = seq[i] + '\n'
    file.write(s)
file.close()
print("保存成功") 

Adding 265 Ns to chr4_JH584295_random:1163-1976


保存成功


<h2> Integrate <h2>

In [5]:
index = ATAC.var_names

index = pd.DataFrame(chr_split(x) for x in index)

index.to_csv('/home/nas2/biod/machuanlong/data/integrate/ATAC.bed',sep='\t',header=False, index=False)

H3K27ac = sc.read_h5ad('/home/nas2/biod/machuanlong/data/P22/H3K27ac/adata_peaks_normalized.h5ad')

index = H3K27ac.var_names

index = pd.DataFrame(chr_split(x) for x in index)

index.to_csv('/home/nas2/biod/machuanlong/data/integrate/H3K27ac.bed',sep='\t',header=False, index=False)

In [6]:
import pybedtools

ATAC_bed = pybedtools.BedTool('/home/nas2/biod/machuanlong/data/integrate/ATAC.bed')
H3K27ac_bed =   pybedtools.BedTool('/home/nas2/biod/machuanlong/data/integrate/H3K27ac.bed')                  

overlap = ATAC_bed.intersect(H3K27ac_bed,wo=True)

overlap.moveto('/home/nas2/biod/machuanlong/data/integrate/overlap.bed')

df = pd.read_csv('/home/nas2/biod/machuanlong/data/integrate/overlap.bed', sep='\t', header=None)

df['start'] = ''
df['end'] = ''

df['start'] = np.where(df.iloc[:, 1] <= df.iloc[:, 4], df.iloc[:, 1], df.iloc[:, 4])
df['end'] = np.where(df.iloc[:, 2] >= df.iloc[:, 5], df.iloc[:, 2], df.iloc[:, 5])
index = df.loc[:, [0, 'start', 'end']]

seq, _ = SpaDC.make_bed_seqs_from_df(index, '/home/nas2/biod/machuanlong/data/Genome/mm10.fa', 1344)

file = open('/home/nas2/biod/machuanlong/data/integrate/overlap_seqs.txt','a')
file.write('seq\n')
for i in range(len(seq)):
    s = seq[i] + '\n'
    file.write(s)
file.close()
print("保存成功") 

保存成功


In [7]:
import anndata as ad
import scipy.sparse

ATAC_matrix = np.array(ATAC.X.todense().transpose())
H3K27ac_matrix = np.array(H3K27ac.X.todense().transpose())

ATAC_var_name= []
H3K27ac_var_name = []

for i in range(len(df)):
    ATAC_var_name.append('-'.join('%s'%i for i in df.iloc[i, 0:3]))
    H3K27ac_var_name.append('-'.join('%s'%i for i in df.iloc[i, 3:6]))

ATAC_index =  list(ATAC.var_names.get_indexer(ATAC_var_name))
H3K27ac_index  =  list(H3K27ac.var_names.get_indexer(H3K27ac_var_name))

new_matrix = np.hstack((ATAC_matrix[ATAC_index], H3K27ac_matrix[H3K27ac_index]))
new_matrix[np.where(new_matrix != 0)] = 1

new_matrix = scipy.sparse.csr_matrix(new_matrix)

adata = ad.AnnData(new_matrix.transpose())
adata.obs['batch'] = np.array(['0'] * ATAC.n_obs + ['1'] * H3K27ac.n_obs)
adata.var['chr'] = index.loc[:, 0].values
adata.var['start'] = index.loc[:, 'start'].values
adata.var['end'] = index.loc[:, 'end'].values

adata.obs.index = np.concatenate((ATAC.obs_names, H3K27ac.obs_names),axis=0)

sc.write('/home/nas2/biod/machuanlong/data/integrate/integrate.h5ad', adata)